# Regridding NetCDF data in Python

This notebook tutorial was produced by the [Knowledge Systems](eatlas.org.au) team at the [Australian Institute of Marine Science](www.aims.gov.au) to  guide the reader through regridding NetCDF data in Python. Please refer to the article on [Visualising NetCDF data](./visualising-netcdf.ipynb) for a more general introduction to working with NetCDF files. This tutorial will use the [eReefs Hydrodynamic and BioGeoChemical models of the Great Barrier Reef (GBR)](https://ereefs.aims.gov.au/ereefs-aims) dataset.

This notebook can be run directly from Google Colab (if not already) by clicking the "Open in Colab" button below.

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/eatlas/netcdf-python/blob/master/notebooks/visualising-netcdf.ipynb)

### Prepare the environment.

In [ ]:
# Re-install 'shapely' because of a bug that crashes the notebook when plotting a variable on a map.
!pip uninstall shapely --yes
!pip install shapely --no-binary shapely

# Install 3rd party libraries required by the code.
!pip3 install netcdf4 cartopy

# Install regridding modules.
!mkdir regrid
#!curl https://colab.research.google.com/github/eatlas/netcdf-python/blob/master/regrid/builder.py --output regrid/builder.py
#!curl https://colab.research.google.com/github/eatlas/netcdf-python/blob/master/regrid/regridder.py --output regrid/regridder.py

# Enable logging.
import logging
logging.basicConfig(level=logging.DEBUG)

In [ ]:
# Download data
!curl https://dapds00.nci.org.au/thredds/fileServer/fx3/gbr1_2.0/gbr1_simple_2018-01-10.nc --output gbr1_simple_2018-01-10.nc

### Connect to the dataset

In [ ]:
# Import the NetCDF library for accessing and manipulating NetCDF libraries.
import netCDF4 as nc

# Open the Par8 NetCDF file downloaded previously.
dataset = nc.Dataset('gbr1_simple_2018-01-10.nc')

# Print the high-level metadata.
print(dataset)

Some things to notice in the dataset metadata:

__`k(44), j(2389), i(510), time(24)`__
- `k` represents depth, of which there is 44 depths.
- `i` and `j` represent a 2-dimensional grid of latitude and longitude values.
- `time` has 24 hours.

__`variables(dimensions): float64 zc(k)`__ - `zc` is the Dimension Variable for depth `k`.

__`variables(dimensions): float64 longitude(j, i), float64 latitude(j, i)`__ - `longitude` and `latitude` are 2-dimensional variables used to determine the latitude/longitude for a specific cell in the grid.

__`variables(dimensions): float32 temp(time, k, j, i)`__ - 4-dimensional variable containing temperature. Given a specific `time` and depth (`k`), the location of the cell (`j`/`i`) can be determined by looking up the `longitude` and `latitude` variables.

The metadata is also available as a Python Dict for easier processing:

In [ ]:
print(dataset.__dict__)

# As an example.
print('\nmetadata_link: ' + dataset.__dict__['metadata_link'])

### Dimensions
Metadata for all dimensions can be access by looping through all available dimensions.

In [ ]:
for dimension in dataset.dimensions.values():
    print(dimension)

An individual dimension can be access directly:

In [ ]:
print(dataset.dimensions['i'])

### Variables
Metadata for all variables can be accessed similarly.

In [ ]:
for variable in dataset.variables.values():
    print(variable)

And for a specific variable:

In [ ]:
print(dataset['temp'])

Notes about the `temp` data:

__`float32 temp(time, k, j, i)`__ - 4-dimensions, with the order being: `time`, depth (`k`), `j` and then `i` (where `j` and `i` are used to identify `latitude` and `longitude` in a 2-dimensional grid).

__`current shape = (24, 44, 2389, 510)`__ - the size of the data cube is 24 x 44 x 1536 x 1344, which matches the size of the dimensions from earlier.

### Visualise the data

In [ ]:
# Which depths to use?
print(f"Depths are from {dataset['zc'][0]}m to {dataset['zc'][43]}m")
print(f"We will use {dataset['zc'][40]}m")

# Longitude and latitude.
lons = dataset['longitude'][:]
lats = dataset['latitude'][:]

# Temperature at 4pm (zero-based).
temp = dataset['temp'][15,40,:,:]
print(f'Variable shape: {temp.shape}')

Find the minimum and maximum values for the map legend.

In [ ]:
import numpy as np
import math

# Scan the extracted data for the minimum and maximum.
min_value = math.floor(np.amin(temp))
max_value = math.ceil(np.amax(temp))
print(f'min value: {min_value}')
print(f'max_value: {max_value}')

Display the original data on a map.

In [ ]:
import matplotlib.pyplot as plt
import cartopy.crs as ccrs

fig = plt.figure(figsize=(16,8))
ax = plt.axes(projection=ccrs.Robinson())
ax.gridlines(linestyle='--',color='black')
ax.coastlines()
clevs = np.arange(min_value,max_value,1)
plt.contourf(lons, lats, temp, clevs, transform=ccrs.PlateCarree(),cmap=plt.cm.jet)
plt.title(dataset['temp'].__dict__['long_name'], size=14)
cb = plt.colorbar(ax=ax, orientation="vertical", pad=0.02, aspect=16, shrink=0.8)
cb.set_label(dataset['temp'].__dict__['units'],size=12,rotation=90,labelpad=15)
cb.ax.tick_params(labelsize=10)

### Regrid the data

Use the regridding utility provided in this repository to convert the 2-dimensional `latitude` and `longitude` grid to a regular (rectilinear) grid where `latitude` and `longitude` are 1-dimensional.

__WARNING__: Depending on the `resolution` selected, the regridding process can take a very long time (hours) to prepare the mapping (input -> output), however the regridding itself is quite fast after the mappings are cached.

In [ ]:
import regrid.builder as RegridBuilder

regridder = RegridBuilder.build_from_input_grid(lats, lons, resolution=0.1)
regridded_temp = regridder.regrid(temp)

print(f'regridded_temp.shape: {np.shape(regridded_temp)}')

Display the regridded data on a map.

In [ ]:
# Scan the extracted data for the minimum and maximum.
min_value = math.floor(np.amin(temp))
max_value = math.ceil(np.amax(temp))
print(f'min value: {min_value}')
print(f'max_value: {max_value}')

fig = plt.figure(figsize=(16,8))
ax = plt.axes(projection=ccrs.Robinson())
ax.gridlines(linestyle='--',color='black')
ax.coastlines()
clevs = np.arange(min_value,max_value,1)
plt.contourf(regridder.longitude_array, regridder.latitude_array, regridded_temp, clevs, transform=ccrs.PlateCarree(),cmap=plt.cm.jet)
plt.title(dataset['temp'].__dict__['long_name'] + ' (regridded)', size=14)
cb = plt.colorbar(ax=ax, orientation="vertical", pad=0.02, aspect=16, shrink=0.8)
cb.set_label(dataset['temp'].__dict__['units'],size=12,rotation=90,labelpad=15)
cb.ax.tick_params(labelsize=10)


## Links
- [Python Netcdf library](https://unidata.github.io/netcdf4-python/)
- [eReefs Hydrodynamic and BioGeoChemical models of the Great Barrier Reef (GBR)](https://ereefs.aims.gov.au/ereefs-aims)

## Acknowledgements
These tutorials build on ideas and content from the following sources:
- [Read NetCDF Data with Python](https://towardsdatascience.com/read-netcdf-data-with-python-901f7ff61648)
